<a href="https://colab.research.google.com/github/Adramimrls/Adramimrls/blob/main/Metodos_de_Validacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# Abre un explorador para elegir un archivo
uploaded = files.upload()

Saving metodosDeValidacion.csv to metodosDeValidacion.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut
import math
import os

# --------------------------
# Parámetros de la práctica
# --------------------------
CSV_PATH = "metodosDeValidacion.csv"
TEST_SIZE = 0.30       # test 30% => train 70%
K = 6                  # número de pliegues para KFold
BOOTSTRAP_TRAIN_PCT = 0.70
N_BOOTSTRAP = 2
PRINT_HEAD_ROWS = 20
MAX_PRINT_LOO = 50     # si LOO produce > MAX_PRINT_LOO splits, imprimimos los primeros MAX_PRINT_LOO splits completos y guardamos la lista completa en CSV

# --------------------------
# Cargar datos
# --------------------------
df = pd.read_csv(CSV_PATH)
n = df.shape[0]
print(f"\nDataset cargado: {CSV_PATH} (n={n} filas, {df.shape[1]} columnas)\n")

# --------------------------
# 1) Hold-out sin mezclar: train_test_split(shuffle=False)
# --------------------------
print("=== 1) Train_test_Split (train 70% / test 30%) sin mezclar ===")
train_df, test_df = train_test_split(df, test_size=TEST_SIZE, shuffle=False)
print(f"Train shape: {train_df.shape}  |  Test shape: {test_df.shape}\n")

print("Conjunto de entrenamiento (train) - vista previa:")
print(train_df.head(PRINT_HEAD_ROWS))
print("\nConjunto de Prueba (test) - vista previa:")
print(test_df.head(PRINT_HEAD_ROWS))
print("\n---\n")

# Para usar índices originales (del DataFrame completo) en los métodos posteriores:
train_idx_original = train_df.index.to_numpy()  # índices en referencia a df original
m = train_df.shape[0]
print(f"Usaremos sólo el conjunto TRAIN (70%) para los métodos 2, 3 y 4 (m = {m} filas).")

# --------------------------
# 2) K-Fold (k=K) sobre train_df (sin mezclar)
# --------------------------
print(f"\n=== 2) K-Fold (k={K}) aplicado sobre el TRAIN (70%) con KFold(shuffle=False) ===")
kf = KFold(n_splits=K, shuffle=False)

fold_num = 0
for train_rel_idx, val_rel_idx in kf.split(np.arange(m)):
    fold_num += 1
    # mapear índices relativos del train_df a índices originales del df
    train_orig = train_idx_original[train_rel_idx]
    val_orig = train_idx_original[val_rel_idx]

    train_fold = df.loc[train_orig]  # usa .loc con índices originales
    val_fold = df.loc[val_orig]

    print(f"\n--- Fold {fold_num} ---")
    print(f"Índices ORIGINALES de entrenamiento (total {len(train_orig)}): {train_orig.tolist()}")
    print(f"Índices ORIGINALES de validación  (total {len(val_orig)}): {val_orig.tolist()}")
    print("\nConjunto de ENTRENAMIENTO (train_fold):")
    # imprimir todo el conjunto de entrenamiento para este fold
    print(train_fold.to_string(index=True))
    print("\nConjunto de PRUEBA (val_fold):")
    # imprimir todo el conjunto de validación para este fold
    print(val_fold.to_string(index=True))
    print("---")

print("\n--- Fin K-Fold ---\n")

# --------------------------
# 3) Leave-One-Out (LOO) sobre train_df
# --------------------------
print("=== 3) Leave-One-Out (LOOCV) sobre el TRAIN (70%) ===")
loo = LeaveOneOut()
n_splits_loo = m  # LOO en el conjunto train

print(f"Total splits (LOOCV sobre train): {n_splits_loo}")
print(f"[INFO] Los índices informados son los índices ORIGINALES del DataFrame cargado ({CSV_PATH}).")

# Generar lista completa de splits (guardamos índices originales para comparar fácilmente)
loo_splits = []  # lista de tuplas: (train_indices_original_list, val_index_original)
count = 0
for train_rel_idx, val_rel_idx in loo.split(np.arange(m)):
    count += 1
    train_orig = train_idx_original[train_rel_idx].tolist()
    val_orig = int(train_idx_original[val_rel_idx][0])  # un único índice original
    loo_splits.append((train_orig, val_orig))

    # Imprimir conjuntos completos: training (m-1 filas) y test (1 fila)
    if count <= MAX_PRINT_LOO:
        print(f"\nLOO split {count}: validación index ORIGINAL = {val_orig}")
        print(f"Train indices ORIGINAL (size = {len(train_orig)}): {train_orig}")
        print("\nConjunto de ENTRENAMIENTO (train):")
        print(df.loc[train_orig].to_string(index=True))
        print("\nConjunto de PRUEBA (val) - una fila:")
        print(df.loc[[val_orig]].to_string(index=True))
        print("---")
    else:
        # solo almacenamos; no imprimimos para evitar salida descomunal
        pass

if n_splits_loo > MAX_PRINT_LOO:
    print(f"\n[AVISO] Hay {n_splits_loo} splits. Se imprimieron los primeros {MAX_PRINT_LOO} splits completos.")
    # guardar la lista completa en CSV para comparar fuera si es grande
    loo_out_df = pd.DataFrame([{"split": i+1, "train_indices": t, "val_index": v} for i, (t, v) in enumerate(loo_splits)])
    csv_save_path = "loo_splits.csv"
    loo_out_df["train_indices"] = loo_out_df["train_indices"].apply(lambda x: ",".join(map(str, x)))
    loo_out_df.to_csv(csv_save_path, index=False)
    print(f"La lista completa de splits (train_indices, val_index) se guardó en: {csv_save_path}")

print("\n--- Fin LOOCV ---\n")

# --------------------------
# 4) Bootstrap: generar N_BOOTSTRAP conjuntos con entrenamiento size = 70% del TRAIN (con reemplazo)
#    aplicados sobre train_df
# --------------------------
BOOTSTRAP_TRAIN_SIZE = int(math.floor(BOOTSTRAP_TRAIN_PCT * m))
print(f"=== 4) Bootstrap: generar {N_BOOTSTRAP} repeticiones con entrenamiento size = {BOOTSTRAP_TRAIN_SIZE} (70% del TRAIN) (con reemplazo) ===")

if BOOTSTRAP_TRAIN_SIZE < 1:
    raise ValueError("BOOTSTRAP_TRAIN_SIZE calculado es < 1. Revisa el tamaño del dataset y la fracción.")

rng_base = np.random.RandomState(42)
for b in range(N_BOOTSTRAP):
    rng = np.random.RandomState(42 + b)  # reproducible por réplica
    # elegimos posiciones relativas dentro de train (0..m-1) con reemplazo
    sample_rel_idx = rng.choice(np.arange(m), size=BOOTSTRAP_TRAIN_SIZE, replace=True)
    sample_orig_idx = train_idx_original[sample_rel_idx]  # índices originales correspondientes
    # calcular OOB relativo al conjunto train (i.e., dentro de train_df)
    mask_rel = np.ones(m, dtype=bool)
    mask_rel[sample_rel_idx] = False
    oob_rel_idx = np.where(mask_rel)[0]
    oob_orig_idx = train_idx_original[oob_rel_idx]

    train_boot = df.loc[sample_orig_idx]  # entrenamiento bootstrap (con duplicados posibles)
    val_oob = df.loc[oob_orig_idx]        # validación OOB (índices originales)
    print(f"\nBootstrap réplica {b+1}:")
    print(f"Índices de entrenamiento (bootstrap, ORIGINALES, con reemplazo): {sample_orig_idx.tolist()}")
    print("Conjunto de ENTRENAMIENTO (train_boot):")
    print(train_boot.to_string(index=True))
    print("\nConjunto de PRUEBA (OOB) asociado a esta réplica:")
    print(val_oob.to_string(index=True))
    print("---")
print("\n--- Fin de la generación de conjuntos de validación ---")


Dataset cargado: metodosDeValidacion.csv (n=20 filas, 2 columnas)

=== 1) Train_test_Split (train 70% / test 30%) sin mezclar ===
Train shape: (14, 2)  |  Test shape: (6, 2)

Conjunto de entrenamiento (train) - vista previa:
     x   y
0    1   2
1    2   4
2    3   6
3    4   8
4    5  10
5    6  12
6    7  14
7    8  16
8    9  18
9   10  20
10  11  22
11  12  24
12  13  26
13  14  28

Conjunto de Prueba (test) - vista previa:
     x   y
14  15  30
15  16  32
16  17  34
17  18  36
18  19  38
19  20  40

---

Usaremos sólo el conjunto TRAIN (70%) para los métodos 2, 3 y 4 (m = 14 filas).

=== 2) K-Fold (k=6) aplicado sobre el TRAIN (70%) con KFold(shuffle=False) ===

--- Fold 1 ---
Índices ORIGINALES de entrenamiento (total 11): [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
Índices ORIGINALES de validación  (total 3): [0, 1, 2]

Conjunto de ENTRENAMIENTO (train_fold):
     x   y
3    4   8
4    5  10
5    6  12
6    7  14
7    8  16
8    9  18
9   10  20
10  11  22
11  12  24
12  13  26
13  